In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
device = torch.device("cpu")
import torch
import torch.func as TF
from functorch import jacrev,jacfwd
import matplotlib.pyplot as plt
import timeit

# I. Setting different ways of metric computation for the decoder

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 3 * 3 * 32),
            nn.ReLU(True)
        )

        self.unflatten = nn.Unflatten(dim=1, 
        unflattened_size=(32, 3, 3))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, 
            stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, 
            padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, 
            padding=1, output_padding=1)
        )
        
    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x
    
decoder = Decoder(encoded_space_dim = 2,fc2_input_dim=128)

# Send to device
decoder.to(device) 

# Load the parameters of the trained decoder without curvature in Loss func
PATH_dec = '../nn_weights/decoder_conv_autoenc.pt'
decoder.load_state_dict(torch.load(PATH_dec))

# Switch to eval mode
decoder.eval()

In [ ]:
def make_grid(numsteps):
    
    xs = torch.linspace(-1.5, 1.5, steps = numsteps)
    ys = torch.linspace(-1.5, 1.5, steps = numsteps)
    #uniform_grid = torch.cartesian_prod(xs,ys)

    # true grid starts from left bottom corner. x is the first to increase
    tgrid = torch.cartesian_prod(ys, xs)
    tgrid = tgrid.roll(1,1)
    return tgrid

In [ ]:
def metric_fd(u, function=decoder, hx=1e-5, hy=1e-5): #this gives metric at point u with steps hx and hy
    u = u.reshape(-1,2)

    dx = torch.tensor([[0.0 + hx, 0.0]])
    dy = torch.tensor([[0.0, 0.0 + hy]])
    dpsi_over_dx = (function(u + dx) - function(u - dx))/(2*hx)
    dpsi_over_dy = (function(u + dy) - function(u - dy))/(2*hy)

    dpsi_over_dx = torch.flatten(dpsi_over_dx).view(784,1)
    dpsi_over_dy = torch.flatten(dpsi_over_dy).view(784,1)
    
    dpsi = torch.cat((dpsi_over_dx,dpsi_over_dy),dim=-1)
    my_metric = torch.matmul(dpsi.T,dpsi)
    return my_metric

def metric_jacfwd(u, function = decoder):
    u = u.reshape(-1,2)
    jac = jacfwd(function)(u)
    jac = jac.reshape(-1,2)
    metric = torch.matmul(jac.T,jac)
    return metric

def metric_jacrev(u, function = decoder):
    u = u.reshape(-1,2)
    jac = jacrev(function)(u)
    jac = jac.reshape(-1,2)
    metric = torch.matmul(jac.T,jac)
    return metric

metric_fd_vmap = TF.vmap(metric_fd)
metric_jacfwd_vmap = TF.vmap(metric_jacfwd)
metric_jacrev_vmap = TF.vmap(metric_jacrev)


In [ ]:
def compute_metric(function=decoder, batch = None, method ={"name":str, "precision":float}):
    if method["name"]=="metric_jacfwd_vmap":
        return metric_jacfwd_vmap(batch, function=function)
    elif method["name"]=="metric_fd_vmap":
        h = method["precision"]
        with torch.no_grad():
            metric = metric_fd_vmap(batch, function=function, hx=h,hy=h)
        return metric
    elif method["name"]=="metric_jacrev_vmap":
        return metric_jacrev_vmap(batch, function=function)

In [ ]:
# a short demo checking if this works
my_batch = torch.rand(3,2)
example_fd_vmap = compute_metric(decoder, my_batch, method={"name": "metric_fd_vmap", "precision" : 1e-4})
example_jacfwd_vmap = compute_metric(decoder, my_batch, method={"name": "metric_jacfwd_vmap", "precision" : None})
example_jacrev_vmap = compute_metric(decoder, my_batch, method={"name": "metric_jacrev_vmap", "precision" : None})
print("fd+vmap:\n",example_fd_vmap,"\n jacfwd+vmap:\n",example_jacfwd_vmap,"\n jacrev+vmap:\n",example_jacrev_vmap)

# II.Testing: performance plots

In [ ]:
batchsize_array = np.linspace(10,9e+4+10,num=6).astype(int)
batchsize_array


In [ ]:
computation_time_fd =[]
#computation_time_jacrev =[]
computation_time_jacfwd =[]

for i in batchsize_array:
    numsteps = i
    my_batch = torch.rand(i,2)

    with_fd_timer = timeit.timeit(stmt="compute_metric(decoder, my_batch, method={\"name\": \"metric_fd_vmap\", \"precision\" : 1e-4})",number=1,globals=globals())
    with_jacfwd_timer = timeit.timeit(stmt="compute_metric(decoder, my_batch, method={\"name\": \"metric_jacfwd_vmap\", \"precision\" : None})",number=1, globals=globals())
    #with_jacrev_timer = timeit.timeit(stmt="compute_metric(decoder, my_batch, method={\"name\": \"metric_jacrev_vmap\", \"precision\" : None})",number=1, globals=globals())
    #computation_time_jacrev.append(with_jacrev_timer)
    computation_time_fd.append(with_fd_timer)
    computation_time_jacfwd.append(with_jacfwd_timer)

In [ ]:
plt.plot(computation_time_jacfwd,label="vmap+jacfwd")
plt.plot(computation_time_fd,label="vmap+fd")
plt.xticks((batchsize_array-10)/18000, labels=batchsize_array)
plt.title("Comparison of jacfwd and fd")
plt.xlabel("Linear size of the grid")
plt.ylabel("Time in seconds")
plt.legend()
plt.show()

# III. Testing: relative error frobenius norm plot

In [ ]:
numsteps = 100
fd_step = 5e-6 
tgrid = make_grid(numsteps)
metric_on_grid_jacfwd = compute_metric(decoder, tgrid, method={"name": "metric_jacfwd_vmap", "precision" : None})
metric_on_grid_jacfwd = metric_on_grid_jacfwd.detach()
metric_on_grid_fd = compute_metric(decoder, tgrid, method={"name": "metric_fd_vmap", "precision" : fd_step})

In [ ]:
xs = torch.linspace(-1.5, 1.5, steps = numsteps)
ys = torch.linspace(-1.5, 1.5, steps = numsteps)

# Fast computation of Frobenious norm on the grid without borders
Newfrob1 = metric_on_grid_jacfwd.norm(dim=(1,2)).view(numsteps,numsteps)
Newfrob1 = Newfrob1[1:-1,1:-1].transpose(0,1)

Newfrob2 = metric_on_grid_fd.norm(dim=(1,2)).view(numsteps,numsteps)
Newfrob2 = Newfrob2[1:-1,1:-1].transpose(0,1)


In [ ]:
#Heat map of the frobenius norm
h = plt.contourf(xs[1:-1], ys[1:-1], Newfrob1)
plt.title('Heatmap of the Frobenius norm of the metric')
plt.xlabel( "x coordinate")
plt.ylabel( "y coordinate")
plt.axis('scaled')
plt.colorbar(label="Frobenius norm of the metric")
#plt.xlim(-1.5 + mean[0], 1.5 + mean[0])
#plt.ylim(-1.5 + mean[1], 1.5 + mean[1])
plt.show()

In [ ]:
#Heat map of the frobenius norm
h = plt.contourf(xs[1:-1], ys[1:-1], 100*abs(Newfrob1-Newfrob2)/Newfrob1)
plt.title('Heatmap of relative error')
plt.xlabel( "x coordinate")
plt.ylabel( "y coordinate")
plt.axis('scaled')
plt.colorbar(label="Relative error of the metric")
#plt.xlim(-1.5 + mean[0], 1.5 + mean[0])
#plt.ylim(-1.5 + mean[1], 1.5 + mean[1])
plt.show()

# IV. Checking ground truth

In [ ]:
my_fun_constant = nn.Linear(2,784,bias=False)
weights_tensor = torch.rand(784,2)
Q, R = torch.linalg.qr(weights_tensor)
with torch.no_grad():
    my_fun_constant.weight = nn.Parameter(Q)
def my_fun_sphere(u):
    u = u.flatten()
    output = torch.cat((torch.sin(u[1])*torch.cos(u[0]).unsqueeze(0),torch.sin(u[1])*torch.sin(u[0]).unsqueeze(0),torch.cos(u[1]).unsqueeze(0)),dim=-1)
    output = torch.cat((output.unsqueeze(0),torch.zeros(781).unsqueeze(0)),dim=1)
    output = output.flatten()
    return output

In [ ]:
#short demo
my_batch = torch.rand(3,2)
print("constant:\n", compute_metric(my_fun_constant, my_batch, method={"name": "metric_jacfwd_vmap", "precision" : None}),
      "\n spheric coordinates:\n", compute_metric(my_fun_sphere, my_batch, method={"name": "metric_jacfwd_vmap", "precision" : None}))

In [ ]:
# another way of depicting heatmaps
def draw_metric_frobenius_norm(numsteps=100, function=decoder, method ={"name":str, "precision":float}):
    tgrid = make_grid(numsteps)
    metric_on_grid = compute_metric(function, tgrid, method={"name": method["name"], "precision" : method["precision"]})
    Frob_norm_on_grid = metric_on_grid.norm(dim=(1,2)).view(numsteps,numsteps)
    Frob_norm_on_grid = Frob_norm_on_grid[1:-1,1:-1].detach()
    
    fig, ax = plt.subplots()
    im = ax.imshow(Frob_norm_on_grid,origin="lower")

    cbar = ax.figure.colorbar(im)

    ax.set_xticks((Frob_norm_on_grid.shape[0]-1)*(np.linspace(0,1,num=11)),labels=np.linspace(-1.5,1.5,num=11).round(1))
    ax.set_yticks((Frob_norm_on_grid.shape[1]-1)*(np.linspace(0,1,num=11)),labels=np.linspace(-1.5,1.5,num=11).round(1))
    plt.xlabel( "x coordinate")
    plt.ylabel( "y coordinate")
    plt.axis('scaled')

    ax.set_title("Frobenius norm of the metric")
    fig.tight_layout()
    plt.show()
    return plt

In [ ]:
draw_metric_frobenius_norm(100,function=my_fun_constant, method={"name": "metric_jacfwd_vmap", "precision" : None})

In [ ]:
draw_metric_frobenius_norm(100,function=my_fun_sphere, method={"name": "metric_jacfwd_vmap", "precision" : None})

In [ ]:
draw_metric_frobenius_norm(100,function=decoder, method={"name": "metric_jacfwd_vmap", "precision" : None})